In [22]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Reshape, BatchNormalization, Dropout
import cv2
import os


In [23]:


def yolo_filter_boxes(boxes, conf, probs, thresh=0.6):
    """Filters YOLO boxes based on confidence and class scores."""
    
    # Compute box scores
    scores = probs * conf

    # Get class with highest score & corresponding score
    classes = tf.argmax(scores, axis=-1)
    class_scores = tf.reduce_max(scores, axis=-1)

    # Create mask for filtering
    mask = class_scores >= thresh

    # Apply mask
    return tf.boolean_mask(class_scores, mask), tf.boolean_mask(boxes, mask), tf.boolean_mask(classes, mask)


In [24]:
def iou(b1, b2):
    """Compute the Intersection over Union (IoU) between two boxes."""
    
    x1, y1, x2, y2 = b1
    x1_, y1_, x2_, y2_ = b2

    # Compute intersection
    xi1, yi1 = max(x1, x1_), max(y1, y1_)
    xi2, yi2 = min(x2, x2_), min(y2, y2_)
    inter_w, inter_h = max(0, yi2 - yi1), max(0, xi2 - xi1)
    inter_area = inter_w * inter_h

    # Compute union
    area1 = (x2 - x1) * (y2 - y1)
    area2 = (x2_ - x1_) * (y2_ - y1_)
    union_area = area1 + area2 - inter_area

    return inter_area / union_area


In [25]:


def yolo_nms(scores, boxes, classes, max_boxes=10, iou_thresh=0.5):
    """Applies Non-Max Suppression (NMS) to filter boxes."""
    
    max_boxes_tensor = tf.Variable(max_boxes, dtype='int32')

    # Get indices of boxes to keep
    nms_idx = tf.image.non_max_suppression(boxes, scores, max_boxes_tensor, iou_thresh)

    # Select filtered scores, boxes, and classes
    return tf.gather(scores, nms_idx), tf.gather(boxes, nms_idx), tf.gather(classes, nms_idx)


In [26]:

# Set both TensorFlow and NumPy seeds
tf.random.set_seed(1)
np.random.seed(1)

def yolo_boxes_to_corners(xy, wh):
    """Convert YOLO box format (x, y, w, h) to (x1, y1, x2, y2)"""
    min_corner = xy - (wh / 2)
    max_corner = xy + (wh / 2)
    return tf.concat([min_corner, max_corner], axis=-1)

def scale_boxes(boxes, img_shape):
    """Rescale the YOLO boxes to the original image size."""
    height, width = tf.cast(img_shape[0], tf.float32), tf.cast(img_shape[1], tf.float32)
    scale = tf.stack([width, height, width, height])
    return boxes * scale

# Generate YOLO outputs with fixed seed
yolo_outputs = (
    tf.random.normal([19, 19, 5, 2], mean=1, stddev=4),
    tf.random.normal([19, 19, 5, 2], mean=1, stddev=4),
    tf.random.normal([19, 19, 5, 1], mean=1, stddev=4),
    tf.random.normal([19, 19, 5, 80], mean=1, stddev=4)
)

In [27]:


def yolo_eval(outputs, img_shape=(720, 1280), max_boxes=10, score_thresh=0.6, iou_thresh=0.5):
    """Converts YOLO model output to final filtered boxes, scores, and classes."""
    
    box_xy, box_wh, box_conf, box_probs = outputs

    # Convert boxes to corner coordinates
    boxes = yolo_boxes_to_corners(box_xy, box_wh)

    # Apply score filtering
    scores, boxes, classes = yolo_filter_boxes(boxes, box_conf, box_probs, score_thresh)

    # Scale boxes to original image size
    boxes = scale_boxes(boxes, img_shape)

    # Apply non-max suppression
    return yolo_nms(scores, boxes, classes, max_boxes, iou_thresh)


In [28]:
# Run YOLO evaluation
scores, boxes, classes = yolo_eval(yolo_outputs)

# Print results
print("scores[2] =", scores[2].numpy())
print("boxes[2] =", boxes[2].numpy())
print("classes[2] =", classes[2].numpy())
print("scores.shape =", scores.numpy().shape)
print("boxes.shape =", boxes.numpy().shape)
print("classes.shape =", classes.numpy().shape)

scores[2] = 152.68243
boxes[2] = [ 1530.0006   -3760.4392     831.5534     -19.682007]
classes[2] = 25
scores.shape = (10,)
boxes.shape = (10, 4)
classes.shape = (10,)


In [29]:
def preprocess_image(image_path, target_size=(416, 416)):
    image = cv2.imread(image_path)
    if image is None:
        raise ValueError(f"Image not found or corrupted: {image_path}")

    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert to RGB
    image = cv2.resize(image, target_size)  # Resize to YOLO input shape
    image = image / 255.0  # Normalize to [0,1]
    return image


def load_data(image_dir, label_dir, subset_size=None):
    images, labels = [], []

    image_files = sorted([f for f in os.listdir(image_dir) if f.lower().endswith((".jpg", ".png", ".jpeg"))])

    if subset_size:
        image_files = image_files[:subset_size]  # Limit dataset size

    for image_file in image_files:
        image_path = os.path.join(image_dir, image_file)
        label_file = os.path.splitext(image_file)[0] + ".txt"
        label_path = os.path.join(label_dir, label_file)

        image = cv2.imread(image_path)
        if image is None:
            print(f"Skipping {image_file}: Cannot read image.")
            continue

        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert to RGB
        images.append(image_path)

        # Read label file
        label_array = []
        if os.path.exists(label_path):
            with open(label_path, "r") as f:
                for line in f:
                    parts = line.strip().split()
                    if len(parts) == 5:
                        label_array.append([float(x) for x in parts])
                    else:
                        print(f"Warning: Skipping incorrect label format in {label_path}")

        # Append labels with correct shape
        labels.append(np.array(label_array, dtype=np.float32) if label_array else np.zeros((0, 5), dtype=np.float32))

    return images, labels


In [30]:
def yolo_model():
    model = tf.keras.Sequential([
        Conv2D(16, (3,3), activation='relu', padding='same', input_shape=(416, 416, 3)),
        BatchNormalization(),
        MaxPooling2D(2,2),

        Conv2D(32, (3,3), activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling2D(2,2),

        Conv2D(64, (3,3), activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling2D(2,2),

        Flatten(),
        Dense(256, activation="relu"),
        Dropout(0.5),  # Reduce memory usage
        Dense(20 * 5, activation="linear"),  # Output: 20 boxes × 5 values each
        Reshape((20, 5))
    ])
    return model

In [31]:
def yolo_loss(y_true, y_pred):
    """Ensures y_true and y_pred have the same shape before loss computation."""
    y_true = tf.convert_to_tensor(y_true, dtype=tf.float32)
    y_pred = tf.convert_to_tensor(y_pred, dtype=tf.float32)

    print(f"DEBUG: y_true shape: {y_true.shape}, y_pred shape: {y_pred.shape}")  # Debugging

    # Ensure they have the same shape
    y_true = tf.reshape(y_true, tf.shape(y_pred))

    return tf.reduce_mean(tf.square(y_true - y_pred))  # Basic MSE loss (modify for YOLO loss)


In [32]:
def pad_labels(labels, max_boxes=20):
    """
    Pads labels to ensure all samples have the same number of bounding boxes.
    Each label should have shape (num_boxes, 5), and we pad it to (max_boxes, 5).
    """
    padded_labels = np.zeros((max_boxes, 5))  # Default fill with zeros
    num_boxes = min(len(labels), max_boxes)  # Avoid exceeding max_boxes
    padded_labels[:num_boxes] = labels[:num_boxes]  # Copy existing boxes
    return padded_labels

def data_generator(images, labels, batch_size):
    while True:
        batch_images = []
        batch_labels = []

        for _ in range(batch_size):
            try:
                # Select a random image & label
                idx = np.random.randint(0, len(images))
                img_path = images[idx]
                label_data = labels[idx]

                # Load Image
                image = cv2.imread(img_path)
                if image is None:
                    print(f"❌ Skipping {img_path} (Failed to load)")
                    continue  # Skip if image is invalid

                # Resize & Normalize Image
                image = cv2.resize(image, (416, 416))
                image = image / 255.0  # Normalize

                # Ensure labels have correct shape
                label_data = pad_labels(label_data)

                batch_images.append(image)
                batch_labels.append(label_data)

            except Exception as e:
                print(f"⚠️ Error processing {img_path}: {e}")
                continue  # Skip faulty example

        # Ensure we return a full batch
        if len(batch_images) == batch_size:
            yield np.array(batch_images), np.array(batch_labels)


In [33]:

# Load data
image_dir = r"C:\Users\user\Downloads\archive\vehicle dataset\train\images"
label_dir = r"C:\Users\user\Downloads\archive\vehicle dataset\train\labels"
images, labels = load_data(image_dir, label_dir, subset_size=500)

# Initialize Data Generator
batch_size = 8
train_gen = data_generator(images, labels, batch_size)

# Create Model
model = yolo_model()
model.compile(optimizer=Adam(), loss=yolo_loss, metrics=["accuracy"])

# Train Model
history = model.fit(train_gen, steps_per_epoch=len(images) // batch_size, epochs=10)

# Save Model
model.save("yolo_trained_model.h5")
print("✅ Model Training Complete!")

Epoch 1/10
DEBUG: y_true shape: (None, 20, 5), y_pred shape: (None, 20, 5)
DEBUG: y_true shape: (None, 20, 5), y_pred shape: (None, 20, 5)
62/62 ━━━━━━━━━━━━━━━━━━━━ 57s 884ms/step - accuracy: 0.1900 - loss: 39.3255
Epoch 2/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 54s 873ms/step - accuracy: 0.2772 - loss: 3.8755
Epoch 3/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 54s 874ms/step - accuracy: 0.3111 - loss: 0.2221
Epoch 4/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 54s 866ms/step - accuracy: 0.3436 - loss: 0.1420
Epoch 5/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 54s 875ms/step - accuracy: 0.3428 - loss: 0.1355
Epoch 6/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 68s 1s/step - accuracy: 0.3753 - loss: 0.1582
Epoch 7/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 90s 1s/step - accuracy: 0.4434 - loss: 0.1359
Epoch 8/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 89s 1s/step - accuracy: 0.4432 - loss: 0.1707
Epoch 9/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 55s 876ms/step - accuracy: 0.4477 - loss: 0.1725
Epoch 10/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 54s 869ms/step - accuracy: 0.4020 - loss: 0.9058


✅ Model Training Complete!
